In [52]:
import os
import re
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

time: 12.8 ms (started: 2023-02-19 17:05:37 +00:00)


In [53]:
with open('text.txt', 'r') as f:
    lines = f.readlines()

paragraphs = []
current_paragraph = ""

for line in lines:
    if line.startswith(".I"):
        if current_paragraph != "":
            paragraphs.append(current_paragraph)
            current_paragraph = ""
    elif line.startswith(".W"):
        continue
    else:
        current_paragraph += line.strip()
if current_paragraph != "":
    paragraphs.append(current_paragraph)

time: 24.5 ms (started: 2023-02-19 17:05:37 +00:00)


In [54]:
paragraphs[0]= paragraphs[0][5:]

time: 814 µs (started: 2023-02-19 17:05:38 +00:00)


In [55]:
paragraphs[0]

'lation between maternal and fetal plasma levels of glucose and freefatty acids .correlation coefficients have been determined between the levels ofglucose and ffa in maternal and fetal plasma collected at delivery .significant correlations were obtained between the maternal and fetalglucose levels and the maternal and fetal ffa levels . from the size ofthe correlation coefficients and the slopes of regression lines itappears that the fetal plasma glucose level at delivery is very stronglydependent upon the maternal level whereas the fetal ffa level atdelivery is only slightly dependent upon the maternal level .'

time: 4.93 ms (started: 2023-02-19 17:05:38 +00:00)


In [56]:
documents = {}
for i in range(len(paragraphs)):
  documents[i+1] = paragraphs[i]

time: 1.47 ms (started: 2023-02-19 17:05:38 +00:00)


In [57]:
import json
with open('data.txt', 'w') as f:
  json.dump(paragraphs,f)
import json
# with open('data.txt', 'w') as f:
#   file_name = json.load(f)

time: 10.6 ms (started: 2023-02-19 17:05:38 +00:00)


In [58]:

# preprocess the text by tokenizing, removing stopwords, and stemming
def preprocess(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    words = [ps.stem(w) for w in tokens if w.isalpha() and w not in stop_words]
    return words

time: 2.32 ms (started: 2023-02-19 17:05:38 +00:00)


In [59]:
from collections import defaultdict
# build the inverted index
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for doc_id, text in documents.items():
        terms = preprocess(text)
        for term in set(terms):
            df = terms.count(term)
            inverted_index[term].append((doc_id, df))
    return inverted_index

time: 1.33 ms (started: 2023-02-19 17:05:38 +00:00)


In [60]:
inverted_index = build_inverted_index(documents)

time: 1.71 s (started: 2023-02-19 17:05:38 +00:00)


In [61]:
# write the inverted index to disk
def write_inverted_index(inverted_index, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for term, postings in inverted_index.items():
        df = len(postings)
        posting_list = [str(doc_id) + ',' + str(df) for doc_id, df in postings]
        posting_file = os.path.join(output_dir, term + '.txt')
        with open(posting_file, 'w') as f:
            f.write('\n'.join(posting_list))
    dictionary = [(term, len(postings), term + '.txt') for term, postings in inverted_index.items()]
    dictionary_file = os.path.join(output_dir, 'dictionary.txt')
    with open(dictionary_file, 'w') as f:
        for term, df, posting_file in dictionary:
            f.write(f'{term:<20}{df:<5}{posting_file}\n')


time: 2.68 ms (started: 2023-02-19 17:05:39 +00:00)


In [62]:
write_inverted_index(inverted_index,"/content/drive/MyDrive/colab files/Q2P1")

time: 1.48 s (started: 2023-02-19 17:05:39 +00:00)


In [63]:
# read the inverted index from disk
def read_inverted_index(input_dir):
    inverted_index = {}
    dict_file = os.path.join(input_dir, "dictionary.txt")
    with open(dict_file, 'r') as f:
        for line in f:
            term = line[:20].strip()
            try:
                df = int(line[20:25].strip())
            except ValueError:
                # print("Skipping line due to invalid document frequency: {}".format(line))
                continue
            posting_file = line[25:].strip()
            try:
                with open(os.path.join(input_dir, posting_file), 'r') as g:
                    posting_list = []
                    for doc_id in g.read().split(","):
                        doc_id = doc_id.strip()
                        if doc_id.isnumeric():
                            posting_list.append(int(doc_id))
                    inverted_index[term] = (df, posting_list)
            except FileNotFoundError:
                print("Posting file not found: {}".format(posting_file))
                continue
    return inverted_index



time: 2.85 ms (started: 2023-02-19 17:05:41 +00:00)


In [64]:
inverted_index_read = read_inverted_index("/content/drive/MyDrive/colab files/Q2P1")

time: 284 ms (started: 2023-02-19 17:05:41 +00:00)


In [65]:
import math

# compute the TF-IDF score for a term in a document
def tfidf(term, doc_id, inverted_index, N):
    tf = 0
    for d, df in inverted_index[term]:
        if d == doc_id:
            tf = df
            break
    idf = math.log(N / len(inverted_index[term]))
    return tf * idf

# compute the TF-IDF vector for a query
def query_tfidf(query, inverted_index, N):
    query_terms = query.strip().lower().split()
    scores = {}
    for term in query_terms:
        if term in inverted_index:
            for doc_id, _ in inverted_index[term]:
                scores[doc_id] = scores.get(doc_id, 0) + tfidf(term, doc_id, inverted_index, N)
    return scores


time: 7.12 ms (started: 2023-02-19 17:05:41 +00:00)


In [66]:
import time

# define the path to the inverted index
# inverted_index_path = "/path/to/inverted_index"

# # read in the inverted index from disk
# inverted_index = read_inverted_index(inverted_index_path)

# define the total number of documents
N = len(documents)

# define the number of results to return per query
k = 10
queries = [    "paracetamol",    "ibuprofen",    "paracetamol ibuprofen",    "hepatitis",    "virus",    "hepatitis virus",    "fever",    "cold",    "cough",    "sore throat",    "influenza",    "common cold",    "headache",    "muscle ache",    "back pain",    "stomach ache",    "nausea",    "vomiting",    "diarrhea",    "constipation"]

# loop through each query and print the top k results
for query in queries:
    start_time = time.time()
    scores = query_tfidf(query, inverted_index, N)
    results = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
    print(f"Query: {query}")
    for doc_id, score in results:
        print(f"Document {doc_id}: {score}")
    print(f"Time taken: {time.time() - start_time} seconds")
    print()


Query: paracetamol
Time taken: 0.0007581710815429688 seconds

Query: ibuprofen
Time taken: 0.0002765655517578125 seconds

Query: paracetamol ibuprofen
Time taken: 0.00029468536376953125 seconds

Query: hepatitis
Time taken: 0.0019121170043945312 seconds

Query: virus
Document 24: 6.198478716492308
Time taken: 0.0012962818145751953 seconds

Query: hepatitis virus
Document 24: 6.198478716492308
Time taken: 8.130073547363281e-05 seconds

Query: fever
Document 28: 5.5053315359323625
Document 365: 5.5053315359323625
Time taken: 7.104873657226562e-05 seconds

Query: cold
Document 82: 4.812184355372417
Document 83: 4.812184355372417
Document 126: 4.812184355372417
Document 421: 4.812184355372417
Time taken: 9.632110595703125e-05 seconds

Query: cough
Document 372: 6.198478716492308
Time taken: 3.695487976074219e-05 seconds

Query: sore throat
Time taken: 2.2411346435546875e-05 seconds

Query: influenza
Document 59: 11.010663071864725
Document 469: 5.5053315359323625
Time taken: 5.197525024414

In [67]:
import os

# define the path to the inverted index
inverted_index_path = "/content/drive/MyDrive/colab files/Q2P1"

# get the size of each file in the inverted index directory
file_sizes = [os.path.getsize(os.path.join(inverted_index_path, f)) for f in os.listdir(inverted_index_path)]

# compute the size of the dictionary
dictionary_size = sum(20 + 8 + len(item[1]) for item in inverted_index.items())


# compute the total space taken by the index
total_size = sum(file_sizes) + dictionary_size

print(f"Size of dictionary: {dictionary_size} bytes")
print(f"Size of posting lists: {sum(file_sizes)} bytes")
print(f"Total size of index: {total_size} bytes")


Size of dictionary: 303962 bytes
Size of posting lists: 544491 bytes
Total size of index: 848453 bytes
time: 65.5 ms (started: 2023-02-19 17:05:41 +00:00)


https://drive.google.com/drive/folders/10rST5J2oSOJ7hBNu4qBcGFt1f45iLUre?usp=sharing

this is the folder where the posting list is present

# QUESTION 2 PART 2

In [68]:
!pip install nltk
!pip install pandas
!pip install numpy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install ipython-autotime

%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15.5 s (started: 2023-02-19 17:05:41 +00:00)


In [69]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

time: 1.15 ms (started: 2023-02-19 17:05:57 +00:00)


In [70]:
import os
import sys
import math
from collections import defaultdict
#from com.gsu.python.helper.parser import Parser
from os.path import dirname
from os import getcwd

time: 732 µs (started: 2023-02-19 17:05:57 +00:00)


In [71]:
with open(r'text.txt') as f:
    lines = f.readlines()

paragraphs = []
current_paragraph = ""

for line in lines:
    if line.startswith(".I"):
        if current_paragraph != "":
            paragraphs.append(current_paragraph)
            current_paragraph = ""
    elif line.startswith(".W"):
        continue
    else:
        current_paragraph += line.strip()
if current_paragraph != "":
    paragraphs.append(current_paragraph)

time: 17.1 ms (started: 2023-02-19 17:05:57 +00:00)


In [72]:
documents = {}
for i in range(len(paragraphs)):
  documents[i+1] = paragraphs[i]

time: 1.37 ms (started: 2023-02-19 17:05:57 +00:00)


In [73]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

time: 1.21 ms (started: 2023-02-19 17:05:57 +00:00)


In [74]:
def convert_lower_case(data):
    return np.char.lower(data)

time: 758 µs (started: 2023-02-19 17:05:57 +00:00)


In [75]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 2.12 ms (started: 2023-02-19 17:05:57 +00:00)


In [76]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

time: 995 µs (started: 2023-02-19 17:05:57 +00:00)


In [77]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

time: 779 µs (started: 2023-02-19 17:05:57 +00:00)


In [78]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 2.31 ms (started: 2023-02-19 17:05:57 +00:00)


In [79]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

time: 1.49 ms (started: 2023-02-19 17:05:57 +00:00)


In [80]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

time: 864 µs (started: 2023-02-19 17:05:57 +00:00)


In [81]:
def preprocess(data, query):
    if not query:
        data = remove_header(data)        
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    #data = stemming(data)
    return data

time: 924 µs (started: 2023-02-19 17:05:57 +00:00)


In [82]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0
i=1

for d in documents:
    
    preprocessed_text = preprocess(documents[i], False)
    print(i)
    i+=1
    if doc%100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))
    #print(tokens,i)
    
    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]            
            #print('p',p,'token',token)
            k = [a[0] for a in p]
            #print('p',p,'token',token,'a',a,'a[0]',a[0],'k',k)
            #print('doc',doc)
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)

        pos += 1
    doc +=1 

No Header
1
0
No Header
2
No Header
3
No Header
4
No Header
5
No Header
6
No Header
7
No Header
8
No Header
9
No Header
10
No Header
11
No Header
12
No Header
13
No Header
14
No Header
15
No Header
16
No Header
17
No Header
18
No Header
19
No Header
20
No Header
21
No Header
22
No Header
23
No Header
24
No Header
25
No Header
26
No Header
27
No Header
28
No Header
29
No Header
30
No Header
31
No Header
32
No Header
33
No Header
34
No Header
35
No Header
36
No Header
37
No Header
38
No Header
39
No Header
40
No Header
41
No Header
42
No Header
43
No Header
44
No Header
45
No Header
46
No Header
47
No Header
48
No Header
49
No Header
50
No Header
51
No Header
52
No Header
53
No Header
54
No Header
55
No Header
56
No Header
57
No Header
58
No Header
59
No Header
60
No Header
61
No Header
62
No Header
63
No Header
64
No Header
65
No Header
66
No Header
67
No Header
68
No Header
69
No Header
70
No Header
71
No Header
72
No Header
73
No Header
74
No Header
75
No Header
76
No Header
77
No Hea

In [83]:
postings

,creaturesand,pity,asexaggerated,overcompensation,him,personsclose,wishes,toanimals,cruelty,sadism,...,freefatty,glucose,of,levels,plasma,fetal,and,maternal,between,correlation
0,"[[490, {93}]]","[[490, {90}]]","[[490, {89}]]","[[490, {88}]]","[[490, {85}]]","[[490, {83}]]","[[490, {81}]]","[[490, {78}]]","[[490, {77}]]","[[490, {76}]]",...,"[[0, {10}], [323, {19}], [329, {73}]]","[[0, {8, 65}], [4, {39}], [9, {61, 50, 3, 21}]...","[[0, {57, 7}], [1, {1, 226, 8, 202, 234, 44, 1...","[[0, {40, 19, 6, 47}], [1, {7}], [3, {58}], [9...","[[0, {64, 27, 5}], [4, {118, 6}], [5, {109, 28...","[[0, {4, 45, 78, 26, 63}], [1, {14}], [2, {64,...","[[0, {3, 38, 9, 41, 44, 21, 54, 25}], [1, {164...","[[0, {2, 37, 74, 43, 24, 88}], [4, {117, 5}], ...","[[0, {1, 35, 17}], [1, {34}], [28, {244}], [31...","[[0, {0, 12, 52}], [28, {217, 242}], [74, {189..."


time: 340 ms (started: 2023-02-19 17:28:47 +00:00)


In [84]:
frequency

,creaturesand,pity,asexaggerated,overcompensation,him,personsclose,wishes,toanimals,cruelty,sadism,...,freefatty,glucose,of,levels,plasma,fetal,and,maternal,between,correlation
0,1,1,1,1,1,1,1,1,1,1,...,3,16,488,39,27,14,464,11,84,14


time: 89.6 ms (started: 2023-02-19 17:28:47 +00:00)


In [85]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])

time: 1.26 ms (started: 2023-02-19 17:28:47 +00:00)


In [86]:
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

time: 809 µs (started: 2023-02-19 17:28:47 +00:00)


In [87]:
def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init

time: 1.39 ms (started: 2023-02-19 17:28:47 +00:00)


In [88]:
def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

time: 2.03 ms (started: 2023-02-19 17:28:47 +00:00)


In [89]:
def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

time: 2.13 ms (started: 2023-02-19 17:28:47 +00:00)


In [90]:
unique_words=list(postings.columns)

time: 8.24 ms (started: 2023-02-19 17:28:47 +00:00)


In [91]:
%cd "/content/drive/MyDrive/colab files/Q2P1"
os.getcwd()

/content/drive/MyDrive/colab files/Q2P1


'/content/drive/MyDrive/colab files/Q2P1'

time: 10 ms (started: 2023-02-19 17:28:47 +00:00)


In [92]:
i=0
j=1
for word in unique_words:
    with open(word + '.txt' ,'w') as datafile:
        y=(postings.iloc[:, i:j])
        y1=y.iloc[:,0:1]
        string = y1.to_string(header=False, index=False)
        datafile.write(string)
        i=i+1
        j=j+1
        datafile.close()

time: 32.7 s (started: 2023-02-19 17:28:47 +00:00)


In [93]:
term_frequency=list(frequency.columns)
#print(term_frequency)

time: 9.36 ms (started: 2023-02-19 17:29:20 +00:00)


In [94]:
k=0
l=1
freq_list=[]
for freq in term_frequency:
    f=frequency.iloc[:, k:l]
    #print('f',f)
    f1=f.iloc[:,0:1]
    #print('f1',f1)
    f2 = f.to_string(header=False, index=False)
    #print('f2',f2,'k',k)
    #freq_list.insert(k,f2)
    freq_list.append(f2)
    #f1=f.iloc[:,0:1]
    #
    k=k+1
    l=l+1
print('freq_list',freq_list)

freq_list ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2',

# array to store all text file(pointer to posting list=index of array)

In [95]:
p=len(postings.columns)
print('no of text files/unique keywords',p)
#initializing the array with total number of keywords
#array_words =  [[0] * 1] * p
array_words=[p]
k=0
size=0
#unique words is a list containg all keywords
for word in unique_words:
    with open(word + '.txt') as datafile:
        #print(datafile,k)
        for line1 in datafile:
            #print(line1)
            #print('index',k)
            lines1=datafile.readlines()
            #print(lines1)
            array_words.insert(k,line1)
            #print(lines1)
            #x=(array_words[k].__sizeof__)
            #print(x)
            #size=size+x
            #print(array_words,'index',k)    
    datafile.close()
    k=k+1
print('length of array',len(array_words))

no of text files/unique keywords 13431
length of array 13432
time: 420 ms (started: 2023-02-19 17:29:28 +00:00)


In [96]:
dictionary_string=''
dictionary_block=''
term_pointer=[]
def count_characters_in_string(mystring):
    s=0
    x=mystring
    for i in x:
        t=i.split()
        s=s+len(t)            
        #print("The number of characters in this string is:",s)
    return(s)
for word in unique_words:
    dictionary_string=dictionary_string+str(word)
    
    x=count_characters_in_string(word)
    term_pointer.append(x)
    print(x,word,'---------')
    
    dictionary_block=dictionary_block+str(x)+str(word)
print('----Term pointer',term_pointer) 

Streaming output truncated to the last 5000 lines.
4 grew ---------
7 nottake ---------
9 autograft ---------
13 autochthonous ---------
4 back ---------
14 autograftswhen ---------
6 inrats ---------
13 fibrosarcomas ---------
6 pyrene ---------
5 benzo ---------
8 actively ---------
6 itsown ---------
13 immunological ---------
10 ofimmunity ---------
12 growthcurves ---------
19 inoculationalthough ---------
10 tumorwhile ---------
7 earlier ---------
8 inoculum ---------
4 thes ---------
13 ofinoculation ---------
16 intenseantigenic ---------
5 route ---------
14 theintradermal ---------
6 routes ---------
13 fromdifferent ---------
7 massive ---------
12 obtainedwith ---------
12 largenumbers ---------
7 inocula ---------
6 grafts ---------
16 immunityelicited ---------
8 subsided ---------
10 hadlargely ---------
12 afterprimary ---------
20 antigenicstimulation ---------
10 allogeneic ---------
9 parameter ---------
8 kinetics ---------
21 tumourtransplantation ---------
11 was

In [97]:
print(dictionary_string)

...
time: 1.04 ms (started: 2023-02-19 17:29:46 +00:00)


In [98]:
print(dictionary_block)

...
time: 705 µs (started: 2023-02-19 17:29:46 +00:00)


# Dictionary as a string

In [99]:
#freq_list---stores frequency of all keywords
#dictionary_string---Keywords stored as a long string
#pointer to posting list-index pointing array element-array_words(contains posting list)
#unique_words----list contains all 
#term_pointer-has length of no of alphabet in each keyword
pointer_to_posting_list=[]
doc_frequency=[]

for word in unique_words:
    print('word-----',word)
    pointer_to_posting_list.append(unique_words.index(word))
    doc_frequency.append(freq_list[unique_words.index(word)])
print('pointer_to_posting_list--------',pointer_to_posting_list)
print('doc_frequency',doc_frequency)
print('term_pointer',term_pointer)

Streaming output truncated to the last 5000 lines.
word----- autograft
word----- autochthonous
word----- back
word----- autograftswhen
word----- inrats
word----- fibrosarcomas
word----- pyrene
word----- benzo
word----- actively
word----- itsown
word----- immunological
word----- ofimmunity
word----- growthcurves
word----- inoculationalthough
word----- tumorwhile
word----- earlier
word----- inoculum
word----- thes
word----- ofinoculation
word----- intenseantigenic
word----- route
word----- theintradermal
word----- routes
word----- fromdifferent
word----- massive
word----- obtainedwith
word----- largenumbers
word----- inocula
word----- grafts
word----- immunityelicited
word----- subsided
word----- hadlargely
word----- afterprimary
word----- antigenicstimulation
word----- allogeneic
word----- parameter
word----- kinetics
word----- tumourtransplantation
word----- wascombined
word----- injectionof
word----- thethymectomy
word----- transplants
word----- challengedwith
word----- cba
word----- 

# dictionary as a block

In [100]:
#block pointer
block_pointer=[]
block_length=4
index = 0 #or start at 3, pending on where you want to start
while index < len(unique_words) :
    #a= list[index]
    block_pointer.append(index)
    index +=block_length
print(block_pointer)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124, 128, 132, 136, 140, 144, 148, 152, 156, 160, 164, 168, 172, 176, 180, 184, 188, 192, 196, 200, 204, 208, 212, 216, 220, 224, 228, 232, 236, 240, 244, 248, 252, 256, 260, 264, 268, 272, 276, 280, 284, 288, 292, 296, 300, 304, 308, 312, 316, 320, 324, 328, 332, 336, 340, 344, 348, 352, 356, 360, 364, 368, 372, 376, 380, 384, 388, 392, 396, 400, 404, 408, 412, 416, 420, 424, 428, 432, 436, 440, 444, 448, 452, 456, 460, 464, 468, 472, 476, 480, 484, 488, 492, 496, 500, 504, 508, 512, 516, 520, 524, 528, 532, 536, 540, 544, 548, 552, 556, 560, 564, 568, 572, 576, 580, 584, 588, 592, 596, 600, 604, 608, 612, 616, 620, 624, 628, 632, 636, 640, 644, 648, 652, 656, 660, 664, 668, 672, 676, 680, 684, 688, 692, 696, 700, 704, 708, 712, 716, 720, 724, 728, 732, 736, 740, 744, 748, 752, 756, 760, 764, 768, 772, 776, 780, 784, 788, 792, 796, 800, 804, 808, 812, 816, 82

In [101]:
def compress(string):
    index = 0
    compressed = ""
    len_str = len(string)
    while index != len_str:
        count = 1
        while (index < len_str-1) and (string[index] == string[index+1]):
            count = count + 1
            index = index + 1
        if count == 1:
            compressed += str(string[index])
        else:
            compressed += str(string[index]) + str(count)
        index = index + 1
    return compressed
      

string = "pythooonnnpool"
print(compress(string))

pytho3n3po2l
time: 2.4 ms (started: 2023-02-19 17:30:04 +00:00)


In [102]:
dictionary_font_coding=compress(dictionary_block)
print(dictionary_font_coding)

...
time: 132 ms (started: 2023-02-19 17:30:04 +00:00)


In [103]:
def run_query_dictionary_string(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        i=0
        j=term_pointer[i]
        #print("Total Document Mathces", [a[0] for a in postings[query][0]])
        #return [a[0] for a in postings[query][0]]
        x=dictionary_string.find(query,i,i+j)
        print(x)
        #index of matched term
        y=array_words[x]
        print(y,x)
        i+=1
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(query)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    print("Total Document Matches:", init_matches)
    return total_matched_docs

time: 4.17 ms (started: 2023-02-19 17:30:04 +00:00)


In [104]:
def run_query_dictionary_block(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        i=0
        p=0
        z=4*8
        j=0
        while(p<z):
            j+=term_pointer[i]
            p+=1
        #print("Total Document Mathces", [a[0] for a in postings[query][0]])
        #return [a[0] for a in postings[query][0]]
        x=dictionary_string.find(query,i,j)
        print(x)
        #index of matched term
        y=array_words[x]
        print(y,x)
        i+=j
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(query)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    print("Total Document Matches:", init_matches)
    return total_matched_docs

time: 2.75 ms (started: 2023-02-19 17:30:04 +00:00)


In [105]:
def run_query_dictionary_block_font(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        i=0
        p=0
        z=4*8
        j=0
        while(p<z):
            j+=term_pointer[i]
            p+=1
        #print("Total Document Mathces", [a[0] for a in postings[query][0]])
        #return [a[0] for a in postings[query][0]]
        x=dictionary_font_coding.find(query,i,j)
        print(x)
        #index of matched term
        y=array_words[x]
        print(y,x)
        i+=j
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(query)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    print("Total Document Matches:", init_matches)
    return total_matched_docs

time: 4.95 ms (started: 2023-02-19 17:30:04 +00:00)


In [106]:
import sys
a1=sys.getsizeof(dictionary_string)
a2=sys.getsizeof(dictionary_block)
a3=sys.getsizeof(dictionary_font_coding)
b1=len(dictionary_string)
b2=len(dictionary_block)
b3=len(dictionary_font_coding)
print(a1,a2,a3,b1,b2,b3)

135906 156116 156109 135857 156067 156060
time: 4.15 ms (started: 2023-02-19 17:30:04 +00:00)


# Time Taken


# Calculating Time dictionary string

In [107]:
import time
queries = ['therapyled', 'toperpetuate', 'unresolvedgrief', 'itsdisadvantages', 'twopumps', 'substituting','cumstances', 'wasto', 'childand', 'measureto', 'ances', 'disturb', 'psychologicalsymptoms', 'instancethere', 'fathers', 'losttheir', 'boys', 'psychoneuroses', 'delinquency']
start_time = time.time()
for query in queries:
    #start_time = time.time()
    print(f"Query: {query}")
    #run_query(query)
    #print(f"Time taken: {time.time() - start_time} seconds")
    run_query_dictionary_string(query)
print(f" TIME TAKEN String Time taken: {time.time() - start_time} seconds")

Query: therapyled
therapyled
['therapyled']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 161)]
Query: toperpetuate
toperpetuate
['toperpetuate']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 147)]
Query: unresolvedgrief
unresolvedgrief
['unresolvedgrief']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 140)]
Query: itsdisadvantages
itsdisadvantages
['itsdisadvantages']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(421, 97)]
Query: twopumps
twopumps
['twopumps']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(421, 84)]
Query: substituting
substituting
['substituting']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(421, 83)]
Query: cumstances
cumstances
['cumstances']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 127)]
Query: wasto
wasto
['wasto']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 1

# Calculating Time dictionary Block

In [108]:
import time
queries = ['therapyled', 'toperpetuate', 'unresolvedgrief', 'itsdisadvantages', 'twopumps', 'substituting','cumstances', 'wasto', 'childand', 'measureto', 'ances', 'disturb', 'psychologicalsymptoms', 'instancethere', 'fathers', 'losttheir', 'boys', 'psychoneuroses', 'delinquency']
start_time = time.time()
for query in queries:
    #start_time = time.time()
    print('-----------------BLOCK------------')
    print(f"Query: {query,start_time}")
    run_query_dictionary_block(query)
    print(time.time())
print(f" TIME TAKEN Block Time taken: {time.time() - start_time} seconds")

-----------------BLOCK------------
Query: ('therapyled', 1676827805.0291107)
therapyled
['therapyled']
358
[[479, {18}]] 358
Total Document Matches: set()
Total Document Matches: [(489, 161)]
1676827805.030365
-----------------BLOCK------------
Query: ('toperpetuate', 1676827805.0291107)
toperpetuate
['toperpetuate']
368
[[478, {50}]] 368
Total Document Matches: set()
Total Document Matches: [(489, 147)]
1676827805.0312746
-----------------BLOCK------------
Query: ('unresolvedgrief', 1676827805.0291107)
unresolvedgrief
['unresolvedgrief']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 140)]
1676827805.0423806
-----------------BLOCK------------
Query: ('itsdisadvantages', 1676827805.0291107)
itsdisadvantages
['itsdisadvantages']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(421, 97)]
1676827805.0435266
-----------------BLOCK------------
Query: ('twopumps', 1676827805.0291107)
twopumps
['twopumps']
-1
13431 -1
Total Document Matches: set()


# Time for FONT CODE for 20 qurries

In [109]:
import time
queries = ['therapyled', 'toperpetuate', 'unresolvedgrief', 'itsdisadvantages', 'twopumps', 'substituting','cumstances', 'wasto', 'childand', 'measureto', 'ances', 'disturb', 'psychologicalsymptoms', 'instancethere', 'fathers', 'losttheir', 'boys', 'psychoneuroses', 'delinquency']

start_time = time.time()
print('START TIME',start_time)
for query in queries:
    #start_time = time.time()
    
    print('-----------------BLOCK------------')
    print(f"Query: {query,start_time}")
    run_query_dictionary_block_font(query)
    print('END TIME',time.time())
print(f" TIME TAKEN Block FONT CODING  Time taken: {time.time() - start_time} seconds")

START TIME 1676827805.18231
-----------------BLOCK------------
Query: ('therapyled', 1676827805.18231)
therapyled
['therapyled']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 161)]
END TIME 1676827805.1870368
-----------------BLOCK------------
Query: ('toperpetuate', 1676827805.18231)
toperpetuate
['toperpetuate']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 147)]
END TIME 1676827805.1933389
-----------------BLOCK------------
Query: ('unresolvedgrief', 1676827805.18231)
unresolvedgrief
['unresolvedgrief']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(489, 140)]
END TIME 1676827805.2036633
-----------------BLOCK------------
Query: ('itsdisadvantages', 1676827805.18231)
itsdisadvantages
['itsdisadvantages']
-1
13431 -1
Total Document Matches: set()
Total Document Matches: [(421, 97)]
END TIME 1676827805.2050054
-----------------BLOCK------------
Query: ('twopumps', 1676827805.18231)
twopumps
['twopumps']
-1
1343

# Space Calculation

In [110]:
import os
# define the path to the inverted index
#inverted_index_path = "/content/drive/MyDrive/colab files/Q2P1"

# get the size of each file in the inverted index directory
#file_sizes = [os.path.getsize(os.path.join(inverted_index_path, f)) for f in os.listdir(inverted_index_path)]

# compute the size of the dictionary
#dictionary_size = sum(20 + 8 + len(item[1]) for item in inverted_index.items())


# compute the total space taken by the index
#total_size = sum(file_sizes) + dictionary_size

#print(f"Size of dictionary: {dictionary_size} bytes")
#print(f"Size of posting lists: {sum(file_sizes)} bytes")
#print(f"Total size of index: {total_size} bytes")

#-------------------
#get the size of dictionary as a string
import sys
#k=block size
k=4
#size of freq term=4 bytes
#size of term pointer=4 bytes
a=sys.getsizeof(dictionary_string)
print('Size of Dictionary stored as String',a)
b=sys.getsizeof(dictionary_block)
print('Size Of Dictionary with Block of size 4',b)
x=sys.getsizeof(dictionary_font_coding)
print('Size Of Dictionary Block Font Coding',x)
c=sys.getsizeof(array_words)
c1=len(postings)%k
total_items=len(postings.columns)

d=total_items*8

d1=total_items*4+(c1*4)

dictionary_size_string=0
dictionary_size_string=(a+d+c)
print('Calculating Size--------------------')
print(f"Size of dictionary as a String in Bytes: {dictionary_size_string} bytes")
dictionary_size_block=(b+d1+c)
dictionary_size_font_coding=(x+d1+c)
print(f"Size of dictionary with block in Bytes: {dictionary_size_block} bytes")
print(f"Size of dictionary with Font Coding in Bytes: {dictionary_size_font_coding} bytes")


Size of Dictionary stored as String 135906
Size Of Dictionary with Block of size 4 156116
Size Of Dictionary Block Font Coding 156109
Calculating Size--------------------
Size of dictionary as a String in Bytes: 356594 bytes
Size of dictionary with block in Bytes: 323084 bytes
Size of dictionary with Font Coding in Bytes: 323077 bytes
time: 5.86 ms (started: 2023-02-19 17:30:05 +00:00)


time: 14.5 ms (started: 2023-02-19 17:30:05 +00:00)
